In [1]:
# Churn Prediction Model using Machine Learning
# This script loads the dataset, preprocesses the data, trains multiple models, and evaluates performance.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [14]:
# Load dataset
df = pd.read_csv('Churn_Modelling.csv')  # Replace with actual dataset path
print("Dataset loaded successfully.")

Dataset loaded successfully.


In [15]:
# Display basic information about the dataset and we see no null values
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB
None


In [16]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'],axis = 1)

In [17]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Encoding categorical data

In [19]:
df = pd.get_dummies(df, drop_first=True).astype(int)

In [20]:
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0,1,1,1,101348,1,0,0,0
1,608,41,1,83807,1,0,1,112542,0,0,1,0
2,502,42,8,159660,3,1,0,113931,1,0,0,0
3,699,39,1,0,2,0,0,93826,0,0,0,0
4,850,43,2,125510,1,1,1,79084,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0,2,1,0,96270,0,0,0,1
9996,516,35,10,57369,1,1,1,101699,0,0,0,1
9997,709,36,7,0,1,0,1,42085,1,0,0,0
9998,772,42,3,75075,2,1,0,92888,1,1,0,1


In [22]:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

### This is imbalanced dataset so we will do oversmapling by SMOTE

In [24]:
X = df.drop('Exited',axis = 1)
y = df['Exited']

In [25]:
from imblearn.over_sampling import SMOTE

In [26]:
X_res, y_res = SMOTE().fit_resample(X,y)

In [27]:
y_res.value_counts()

Exited
1    7963
0    7963
Name: count, dtype: int64

### Train_Test_Split

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.2, random_state = 42)

### Feature Scaling

In [32]:
from sklearn.preprocessing import StandardScaler    ## as we see credit score value and age huge difference 

In [33]:
sc = StandardScaler()

In [34]:
X_train = sc.fit_transform(X_train)
X_test  = sc.transform (X_test)


In [35]:
X_train

array([[-0.85765788,  0.80555978, -0.29269913, ..., -0.57287883,
        -0.46150489, -0.84688403],
       [ 1.08653534,  0.10281864, -1.39388635, ..., -0.57287883,
        -0.46150489,  1.18079922],
       [ 0.66294017,  0.50438501,  1.17555048, ...,  1.74556981,
        -0.46150489, -0.84688403],
       ...,
       [ 0.19589934,  0.20321023,  1.17555048, ..., -0.57287883,
        -0.46150489,  1.18079922],
       [ 0.29365207,  0.20321023,  0.07436327, ..., -0.57287883,
        -0.46150489, -0.84688403],
       [ 0.52174178,  1.00634296, -1.39388635, ...,  1.74556981,
        -0.46150489,  1.18079922]])

## Now will try different models

### 1 ) Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
log = LogisticRegression()

In [38]:
log.fit(X_train, y_train)

LogisticRegression()

In [39]:
y_pred1 = log.predict(X_test)

In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [43]:
print('Accuracy =', accuracy_score(y_test, y_pred1))
print('Precision =', precision_score(y_test, y_pred1))
print('Recall =', recall_score(y_test, y_pred1))
print('F1_score =', f1_score(y_test, y_pred1))

Accuracy = 0.7912743251726303
Precision = 0.7764632627646326
Recall = 0.8029620090148101
F1_score = 0.7894903450459007


### 2 ) SVM

In [44]:
from sklearn import svm

In [46]:
svm = svm.SVC()

In [47]:
svm.fit(X_train, y_train)

SVC()

In [48]:
y_pred2 = svm.predict(X_test)

In [49]:
print('Accuracy =', accuracy_score(y_test, y_pred2))
print('Precision =', precision_score(y_test, y_pred2))
print('Recall =', recall_score(y_test, y_pred2))
print('F1_score =', f1_score(y_test, y_pred2))

Accuracy = 0.8427495291902072
Precision = 0.8291614518147684
Recall = 0.8531873792659369
F1_score = 0.8410028562361155


### 3 ) KNN

In [50]:
from sklearn.neighbors import KNeighborsClassifier

In [51]:
knn = KNeighborsClassifier()

In [52]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [53]:
y_pred3 = knn.predict(X_test)

In [54]:
print('Accuracy =', accuracy_score(y_test, y_pred3))
print('Precision =', precision_score(y_test, y_pred3))
print('Recall =', recall_score(y_test, y_pred3))
print('F1_score =', f1_score(y_test, y_pred3))

Accuracy = 0.8220338983050848
Precision = 0.8002436053593179
Recall = 0.8461043142305216
F1_score = 0.8225352112676056


### 4 ) Decision Trees Classifier

In [55]:
from sklearn.tree import DecisionTreeClassifier

In [56]:
dt = DecisionTreeClassifier()

In [57]:
dt.fit(X_train,y_train)

DecisionTreeClassifier()

In [58]:
y_pred4 = dt.predict(X_test)

In [59]:
print('Accuracy =', accuracy_score(y_test, y_pred4))
print('Precision =', precision_score(y_test, y_pred4))
print('Recall =', recall_score(y_test, y_pred4))
print('F1_score =', f1_score(y_test, y_pred4))

Accuracy = 0.7884494664155681
Precision = 0.7707948243992606
Recall = 0.8055376690276883
F1_score = 0.7877833753148614


### 5) Random Forest

In [60]:
from sklearn.ensemble import RandomForestClassifier

In [61]:
rf = RandomForestClassifier()

In [62]:
rf.fit(X_train, y_train)

RandomForestClassifier()

In [63]:
y_pred5 = rf.predict(X_test)

In [64]:
print('Accuracy =', accuracy_score(y_test, y_pred5))
print('Precision =', precision_score(y_test, y_pred5))
print('Recall =', recall_score(y_test, y_pred5))
print('F1_score =', f1_score(y_test, y_pred5))

Accuracy = 0.8628374136848713
Precision = 0.8522727272727273
Recall = 0.8692852543464262
F1_score = 0.8606949314631814


### 6 ) Gradient Boosting Classifier

In [65]:
from sklearn.ensemble import GradientBoostingClassifier

In [66]:
 gbc = GradientBoostingClassifier()

In [67]:
gbc.fit(X_train, y_train)

GradientBoostingClassifier()

In [68]:
y_pred6 = gbc.predict(X_test)

In [69]:
print('Accuracy =', accuracy_score(y_test, y_pred6))
print('Precision =', precision_score(y_test, y_pred6))
print('Recall =', recall_score(y_test, y_pred6))
print('F1_score =', f1_score(y_test, y_pred6))

Accuracy = 0.8462021343377275
Precision = 0.8378893833439288
Recall = 0.8486799742433999
F1_score = 0.8432501599488164


In [72]:
final_data = pd.DataFrame({'Models' : ['LR','SVC','KNN','DT','RF','GBC'],
                          'Accuracy' : [accuracy_score(y_test, y_pred1),
                                  accuracy_score(y_test, y_pred2),
                                  accuracy_score(y_test, y_pred3),
                                  accuracy_score(y_test, y_pred4),
                                  accuracy_score(y_test, y_pred5),
                                  accuracy_score(y_test, y_pred6)],
                           'Precision' : [precision_score(y_test, y_pred1),
                                  precision_score(y_test, y_pred2),
                                  precision_score(y_test, y_pred3),
                                  precision_score(y_test, y_pred4),
                                  precision_score(y_test, y_pred5),
                                  precision_score(y_test, y_pred6)],
                            'Recall' : [recall_score(y_test, y_pred1),
                                  recall_score(y_test, y_pred2),
                                  recall_score(y_test, y_pred3),
                                  recall_score(y_test, y_pred4),
                                  recall_score(y_test, y_pred5),
                                  recall_score(y_test, y_pred6)],
                            'F1 Score' : [f1_score(y_test, y_pred1),
                                  f1_score(y_test, y_pred2),
                                  f1_score(y_test, y_pred3),
                                  f1_score(y_test, y_pred4),
                                  f1_score(y_test, y_pred5),
                                  f1_score(y_test, y_pred6)]
                                  })

In [73]:
final_data

,Models,Accuracy,Precision,Recall,F1 Score
0,LR,0.791274,0.776463,0.802962,0.789490
1,SVC,0.842750,0.829161,0.853187,0.841003
2,KNN,0.822034,0.800244,0.846104,0.822535
3,DT,0.788449,0.770795,0.805538,0.787783
4,RF,0.862837,0.852273,0.869285,0.860695
5,GBC,0.846202,0.837889,0.848680,0.843250


## Here wee see the best model to for prediction is Random Forest 

In [2]:
# train_churn_model.py

import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib  # pip install joblib

# 1. Load dataset
df = pd.read_csv("Churn_Modelling.csv")

# 2. Drop useless columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# 3. One-hot encode categorical features (same as your notebook)
df = pd.get_dummies(df, drop_first=True).astype(int)

# 4. Split features & target
X = df.drop('Exited', axis=1)
y = df['Exited']

# 5. Handle class imbalance with SMOTE
X_res, y_res = SMOTE().fit_resample(X, y)

# 6. Train-Test split
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42
)

# 7. Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 8. Train best model = RandomForest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled, y_train)

# 9. Quick evaluation (optional)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = rf.predict(X_test_scaled)
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1-score :", f1_score(y_test, y_pred))

# 10. Save everything needed for inference
artifacts = {
    "model": rf,
    "scaler": scaler,
    "feature_columns": X.columns.tolist()  # order of columns after get_dummies
}

joblib.dump(artifacts, "churn_model_rf.joblib")
print("✅ Model saved to churn_model_rf.joblib")


Accuracy : 0.8618957940991839
Precision: 0.8471615720524017
Recall   : 0.8744365743721829
F1-score : 0.8605830164765526
✅ Model saved to churn_model_rf.joblib
